
<a href="https://github.com/dai-zz-flora/dai-zz-flora.github.com/blob/master/chat_with_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I. 准备工作

In [ ]:
# @title ### 1.1 安装依赖
%env TF_CPP_MIN_LOG_LEVEL=1

!apt -y update -qq
!apt -y install -qq aria2
!pip install -q transformers datasets  sentencepiece cpm_kernels mdtex2html accelerate sentencepiece install faiss-gpu  sentence-transformers  PyPDF2 pdfplumber sacremoses

In [ ]:
# @title ## 1.2. 指定数据集


# @markdown ### 本地路径
data_name = "/content/sample_data/california_housing_train.csv"  # @param ["", "/content/sample_data/california_housing_train.csv", "/content/sample_data/mnist_test.csv"]
# @markdown ### 下载路径，格式为csv后缀
download_url = None  # @param 

data_file_name = ""
if download_url:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{download_url}" -d /content/sample_data -o download_data.csv
  data_file_name = "/content/sample_data/download_data.csv"
else:
  data_file_name = data_name




In [ ]:
# @title ## 1.3 定义工具函数
def extract_text_between_a_and_b(text, a, b):
  start_index = text.rfind(a)
  end_index = text.find(b,start_index+3)
    
  if start_index == -1 or end_index == -1:
    return None
    
  return text[start_index + len(a):end_index]

# II. 用THUDM/chatglm-6b来生成代码

In [ ]:
# @title # 2.1. 生成问题描述输入框
from ipywidgets import widgets
from IPython.display import display

textarea = widgets.Textarea(
    value='新生成一列total，值等于每一行的total_bedrooms列和population列相乘,打印total列',  # 填入默认值，可以为空
    placeholder='问题描述',  # 在文本框中显示的提示文本
    description='问题',  # 描述文本，显示在文本框上方
    disabled=False , # 是否禁用
     layout=widgets.Layout(width='800px', height='300px')
)

 
 
display(widgets.HBox([textarea,]))

In [ ]:
# @title ## 2.2. 生成prompt
PROMPT_TEMPLATE = """
写一段python代码，利用pandas库来解决下面的问题:
读取文件 {0} 到一个叫df的DataFrame变量，对df进行操作，{1}



"""
code_desc = textarea.value



prompt = PROMPT_TEMPLATE.format(data_file_name,code_desc)
print(prompt)

In [ ]:
# @title ## 2.3. 初始化模型，为了节省内存使用chatglm-6b-int4

from transformers import AutoTokenizer, AutoModel
checkpoint = "THUDM/chatglm-6b-int4"
code_tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
code_model = AutoModel.from_pretrained(checkpoint, trust_remote_code=True).half().cuda()


In [ ]:
# @title ## 2.4. 生成回答，从回答中截取代码

code_model = code_model.eval()
response, history = code_model.chat(code_tokenizer, prompt, history=[])
# 
result_code = extract_text_between_a_and_b(response,"```python","```")
print("---- 这是原回答")
print(response)
print("\n\n--- 这是截取的代码")
print(result_code)

In [ ]:
# @title ## 2.4. 执行代码，并拿到局部变量的
exec(result_code,locals())
df = locals()["df"]
